In [95]:
import Graphs;
import numpy as np;
import math;

In [96]:
class WeightedVertex(Graphs.Vertex):
    
    def fields(self):
        super().fields();  
        self.operation = None;
        self.area = 0;
    
    def __init__(self, ID, area, operation, graph):
        super().__init__(ID, graph);
        self.area = area;
        self.operation = operation;
    
    def search(self, endCycle = None):
        
        maxVal = 0;
        #delay = 0;
        if(endCycle != None): #ALAP
            #if(self.graph.limit != None):
            #    while( (self.graph.calcAreaAtCycle(endCycle)+self.area) > self.graph.limit):
            #        delay += 1;
            #        endCycle -= 1;
            #    #print("The area is OK! And its value is " + str(self.graph.calcAreaAtCycle(endCycle) + self.area ));
            self.graph.starts.append(endCycle);
            self.graph.startOps.append(self.area);

        self.graph.prenum[self.ID] = self.graph.prenumID;
        self.graph.prenumID += 1;

        if(len(self.directions) == 0):
            #print("Dead End! at [" + str(self.ID) + "]");
            pass;
        else:
            i = 0;
            for d in self.directions:      
                #print("Now in " + str(self.ID) + " going into " + str(d.ID))

                if(self.graph.prenum[d.ID] == 0):
                    self.graph.addEdge(self, d, Graphs.EdgeType.TREE);
                    #print("Going to " + str(d.ID) + "; Val is " + str(val));
                    if(endCycle == None): #ASAP
                        val = d.search();
                    else: #ALAP
                        val = d.search(endCycle-1);
                    if(val > maxVal):
                         maxVal = val;
                    #print("Returned from " + str(d.ID) + "; Val is " + str(val) + " while max is " + str(maxVal));
                else:
                   # print("Already visited " + str(d.ID));
                    self.revisit(d);
                i += 1;
        
        self.graph.postnum[self.ID] = self.graph.postnumID;
        self.graph.postnumID += 1;    

        if(endCycle == None): #ASAP
            #if(self.graph.limit != None):
            #    while( (self.graph.calcAreaAtCycle(maxVal+delay) + self.area) > self.graph.limit ):
            #        delay += 1;
            self.graph.starts.append(maxVal);
            self.graph.startOps.append(self.area);
            
        #maxVal += delay;
        return maxVal+1;

In [ ]:
class SchedulingGraph(Graphs.Graph):
     
    def fields(self):
        super().fields();
        self.starts = [];
        self.startOps = [];
        self.limit = None;
    
    def __init__(self, filename):
        self.fields();
        tV = SchedulingGraph.load_graph(filename);
        for i in range(len(tV)):
            self.vertices.append(WeightedVertex(tV[i][0], tV[i][1], tV[i][2], self));
        maxedge = len(self.vertices);

        self.prenum = np.full(maxedge, 0);
        self.postnum = np.full(maxedge, 0);
        
        for i in range(len(tV)):
            for v in self.vertices:
                #print(v.ID);
                #print(tV[i][0]);
                if (v.ID == tV[i][0]):
                    #print("Found match");
                    for dID in tV[i][3]:
                         for vD in self.vertices:
                            if (vD.ID == dID):
                                #print("Found Destination match");
                                v.addDirection(vD);
                    break;
    
    @staticmethod
    def load_graph(path):
        
        vertices = []; # {{id, area {destination,...}},...}
        
        with open(path, 'r', encoding='utf-8', errors='ignore') as g_file:
            next(g_file); # skip the header line
            
            for line in g_file:
                try:
                    fields = line.split(",");
                    vertex = [];
                    vertex.append(int(fields[0]));
                    vertex.append(int(fields[1]));
                    vertex.append(str(fields[2]));
                    destinations = [];
                    if(len(fields) > 3):
                        for el in fields[3:]:
                            if(el != "\n"):
                                destinations.append(int(el));
                    vertex.append(destinations);
                    vertices.append(vertex);
                except Exception as e:
                    pass
            #print(vertices);
        return vertices;
    
    def addLimit(self, operation, number):
        if(self.limit == None):
            self.limit = [];
        self.limit.append((operation, int(number)));
    
    def clearLimit(self):
        self.limit = None;
        
    def startSearchAt(self, vertexID, ASAP = True):
        #clear previous results
        self.prenum = np.full(len(self.vertices), 0);
        self.postnum = np.full(len(self.vertices), len(self.vertices)); 
        self.prenumID = 1;
        self.posnumID = 1;
        self.edges = [];
        self.starts = [];
        self.startOps = [];
        #and start the search
        if(ASAP):
            return self.vertices[vertexID].search();
        else:
            return self.vertices[vertexID].search(0);
    
    def ASAP(self, vertexID = 0):
        print("Took " + str(self.startSearchAt(vertexID, True)) + " cycles");
        #self.calcArea();
        print("\n\n");
    
    def ALAP(self, vertexID = 0):
        print("Took " + str(self.startSearchAt(vertexID, False)) + " cycles");
        self.calcArea();
        print("\n\n");
    
    def calcArea(self):
        print("Time slices of concurrent operations");
        print(self.starts);
        print(self.startOps);
        
        concArea = [];
        concOps = [];
        for i in range(len(self.starts)):
            found = False;
            opIndex = 0;
            for op in concOps:
                if(op == self.starts[i]):
                    found = True;
                    break;
                opIndex += 1;
            if (not found):
                concOps.append(self.starts[i]);
                concArea.append(self.startOps[i]);
            else:
                concArea[opIndex] += self.startOps[i];
        print("Concurrent ops: ");
        print(concArea);
        print("Maximum needed area: " + str(max(concArea)));
        print("Area constrain: " + str(self.limit));
    
    def isConstrained(self, cycle, operation):
        nOps = 1;
        for i in range(len(self.starts)):
            if(self.starts[i] == cycle):
                if(startOps == operation):
                    nOps += 1;
        for tupl in self.limit:
            if(tupl[0] == operation):
                if(tupl[1] < nOps):
                    return False; #can't add
                else:
                    return True; #can add
        

In [ ]:
g2 = SchedulingGraph("nodesOps.txt");
#g2.ASAP();
g2.ALAP();
g2.clearLimit();
g2.addLimit("MULT", 2);
g2.addLimit("ADD", 2);
print(g2.limit);
#g2.ASAP();
#g2.ALAP();